# CRS Canada immigration metrics analysis
We investigate the Comprehensive Ranking Systme (CRS) metrics that canada uses to grant permanent residence invitations and project predicted cutoff values for the incoming cohort.

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [52]:
url = 'https://www.canada.ca/content/dam/ircc/documents/json/ee_rounds_123_en.json'
r = requests.get(url)
rounds = r.json()['rounds']

## Creating the pandas dataframe

In [69]:
df = pd.DataFrame.from_dict(rounds)
df= df.drop(columns=['drawNumberURL','mitext','DrawText1','drawDistributionAsOn','drawDateFull'])

In [70]:
columns={"drawNumber": "id", 
         "drawDate": "date", 
         "drawName": "round type", 
         "drawSize":"invitations issued", 
         "drawCRS": "CRS cutoff",
         "drawText2": "type issued",
         "drawCutOff": "tie break rule",
         "dd18": "total applications",
         "dd1":  "crs_range_601_1200",
         "dd2":  "crs_range_501_600",
         "dd3":  "crs_range_451_500",
         "dd4":  "crs_range_491_500",
         "dd5":  "crs_range_481_490",
         "dd6":  "crs_range_471_480",
         "dd7":  "crs_range_461_470",
         "dd8":  "crs_range_451_460",
         "dd9":  "crs_range_401_450",
         "dd10": "crs_range_441_450",
         "dd11": "crs_range_431_440",
         "dd12": "crs_range_421_430",
         "dd13": "crs_range_411_420",
         "dd14": "crs_range_401_410",
         "dd15": "crs_range_351_400",
         "dd16": "crs_range_301_350",
         "dd17": "crs_range_000_300",
         }
df.rename(columns=columns)


,id,date,round type,invitations issued,CRS cutoff,type issued,drawDateTime,tie break rule,crs_range_601_1200,crs_range_501_600,...,crs_range_401_450,crs_range_441_450,crs_range_431_440,crs_range_421_430,crs_range_411_420,crs_range_401_410,crs_range_351_400,crs_range_301_350,crs_range_000_300,total applications
0,302,2024-07-08,French language proficiency (Version 1),"3,200",420,"Federal Skilled Worker Program, Canadian Exper...","July 08, 2024 at 14:01:33 UTC","May 30, 2024 at 03:37:43 UTC",940,"16,521",...,"54,148","11,031","11,342","9,947","10,876","10,952","54,902","25,752","5,523","222,547"
1,301,2024-07-05,Healthcare occupations (Version 1),"3,750",445,"Federal Skilled Worker Program, Canadian Exper...","July 05, 2024 at 16:03:12 UTC","May 07, 2024 at 21:27:04 UTC",940,"16,521",...,"54,148","11,031","11,342","9,947","10,876","10,952","54,902","25,752","5,523","222,547"
2,300,2024-07-04,Trade occupations (Version 1),"1,800",436,"Federal Skilled Worker Program, Canadian Exper...","July 4, 2024 at 13:30:48 UTC","July 16, 2023 at 15:44:59 UTC",940,"16,521",...,"54,148","11,031","11,342","9,947","10,876","10,952","54,902","25,752","5,523","222,547"
3,299,2024-07-02,Provincial Nominee Program,920,739,Provincial Nominee Program,"July 2, 2024 at 15:12:05 UTC","February 02, 2024 at 11:38:43 UTC",940,"16,521",...,"54,148","11,031","11,342","9,947","10,876","10,952","54,902","25,752","5,523","222,547"
4,298,2024-06-19,Provincial Nominee Program,"1,499",663,Provincial Nominee Program,"June 19, 2024 at 14:37:26 UTC","January 19, 2024 at 10:10:40 UTC","3,002","14,330",...,"54,200","11,026","11,086","9,989","10,927","11,172","57,086","27,247","5,624","224,940"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,5,2015-03-20,No Program Specified,"1,620",481,"Federal Skilled Worker, Canadian Experience Cl...","March 20, 2015 at 23:59:53 UTC",,0,0,...,0,0,0,0,0,0,0,0,0,0
299,4,2015-02-27,No Program Specified,"1,187",735,"Federal Skilled Worker, Canadian Experience Cl...","February 27, 2015 at 23:59:51 UTC",,0,0,...,0,0,0,0,0,0,0,0,0,0
300,3,2015-02-20,Canadian Experience Class,849,808,Canadian Experience Class,"February 20, 2015 at 11:59:47 UTC",,0,0,...,0,0,0,0,0,0,0,0,0,0
301,2,2015-02-07,No Program Specified,779,818,"Federal Skilled Worker, Canadian Experience Cl...","February 7, 2015 at 11:59:59 UTC",,0,0,...,0,0,0,0,0,0,0,0,0,0
